In [0]:
# 1. Definir la configuración usando nuestra nueva estructura de catálogo
catalog_name = "sesion_5"
bronze_schema = "bronze"
silver_schema = "silver"

# 2. Definir la ruta al volumen de la capa Bronze
bronze_volume_path = f"/Volumes/{catalog_name}/{bronze_schema}/raw_files"

# 3. Cargar todos los datasets desde el volumen Bronze en DataFrames
print(f"Leyendo archivos desde: {bronze_volume_path}...")
options = {"header": "true", "inferSchema": "true"}

customers_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_customers_dataset.csv")
order_items_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_order_items_dataset.csv")
order_payments_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_order_payments_dataset.csv")
orders_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_orders_dataset.csv")
products_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_products_dataset.csv")
sellers_df = spark.read.options(**options).csv(f"{bronze_volume_path}/olist_sellers_dataset.csv")
category_translation_df = spark.read.options(**options).csv(f"{bronze_volume_path}/product_category_name_translation.csv")

print("\n¡Carga completada! Los DataFrames de la capa Bronze están en memoria.")

In [0]:
# Paso 1: Registrar el DataFrame como una vista temporal para poder usar SQL sobre él.
customers_df.createOrReplaceTempView("customers_bronze_vw")

print("Vista temporal 'customers_bronze_vw' creada.")

In [0]:

%sql
-- Paso 2: Crear la tabla con el esquema explícito y luego insertar los datos.
-- Este enfoque de dos pasos nos da control total sobre la estructura de la tabla.

-- Primero, creamos la tabla con la columna de la llave primaria definida como NOT NULL.
CREATE OR REPLACE TABLE sesion_5.silver.dim_clientes_sql (
  customer_id STRING NOT NULL,
  customer_unique_id STRING,
  customer_zip_code_prefix STRING,
  customer_city STRING,
  customer_state STRING
)
USING DELTA
COMMENT 'Dimensión de Clientes, creada con SQL y con esquema definido.';

-- Segundo, insertamos los datos en la tabla ya creada.
INSERT INTO sesion_5.silver.dim_clientes_sql
SELECT DISTINCT
  customer_id,
  customer_unique_id,
  customer_zip_code_prefix,
  customer_city,
  customer_state
FROM customers_bronze_vw;


In [0]:
# 1. Registrar los DataFrames necesarios como vistas temporales
products_df.createOrReplaceTempView("products_bronze_vw")
category_translation_df.createOrReplaceTempView("category_translation_bronze_vw")

print("Vistas temporales 'products_bronze_vw' y 'category_translation_bronze_vw' creadas.")

In [0]:
%sql
-- Crear la tabla de dimensión de productos
-- **CORRECCIÓN**: Añadimos la definición de la columna PK como NOT NULL
CREATE OR REPLACE TABLE sesion_5.silver.dim_productos (
  product_id STRING NOT NULL,
  product_category STRING,
  product_photos_qty INT,
  product_weight_g DOUBLE,
  product_length_cm DOUBLE,
  product_height_cm DOUBLE,
  product_width_cm DOUBLE
)
USING DELTA
COMMENT 'Dimensión de Productos, enriquecida.';

-- Insertamos los datos desde nuestra vista temporal
INSERT INTO sesion_5.silver.dim_productos
SELECT DISTINCT
  p.product_id,
  t.product_category_name_english AS product_category,
  p.product_photos_qty,
  p.product_weight_g,
  p.product_length_cm,
  p.product_height_cm,
  p.product_width_cm
FROM products_bronze_vw p
LEFT JOIN category_translation_bronze_vw t ON p.product_category_name = t.product_category_name;

In [0]:
# 1. Registrar el DataFrame de vendedores como una vista temporal
sellers_df.createOrReplaceTempView("sellers_bronze_vw")

print("Vista temporal 'sellers_bronze_vw' creada.")

In [0]:
%sql
-- Crear la tabla de dimensión de vendedores
-- **CORRECCIÓN**: Añadimos la definición de la columna PK como NOT NULL
CREATE OR REPLACE TABLE sesion_5.silver.dim_vendedores (
  seller_id STRING NOT NULL,
  seller_zip_code_prefix STRING,
  seller_city STRING,
  seller_state STRING
)
USING DELTA
COMMENT 'Dimensión de Vendedores, depurada y modelada.';

-- Insertamos los datos
INSERT INTO sesion_5.silver.dim_vendedores
SELECT DISTINCT
  seller_id,
  seller_zip_code_prefix,
  seller_city,
  seller_state
FROM sellers_bronze_vw;

-- Verificamos
SELECT * FROM sesion_5.silver.dim_vendedores LIMIT 5;

In [0]:
# 1. Registrar todos los DataFrames relacionados con los pedidos como vistas temporales
orders_df.createOrReplaceTempView("orders_bronze_vw")
order_items_df.createOrReplaceTempView("order_items_bronze_vw")
order_payments_df.createOrReplaceTempView("order_payments_bronze_vw")

print("Vistas temporales para pedidos, items y pagos creadas.")

In [0]:
%sql
-- Crear la tabla de hechos
-- **CORRECCIÓN**: Definimos las columnas que serán FK como NOT NULL
CREATE OR REPLACE TABLE sesion_5.silver.fact_pedidos (
  order_id STRING,
  customer_id STRING NOT NULL,
  product_id STRING NOT NULL,
  seller_id STRING NOT NULL,
  fecha_pedido DATE,
  order_status STRING,
  price DOUBLE,
  costo_envio DOUBLE,
  secuencia_pago INT,
  tipo_pago STRING,
  cuotas_pago INT,
  valor_pago DOUBLE
)
USING DELTA
COMMENT 'Tabla de hechos con las métricas de los pedidos.';

-- Insertamos los datos
INSERT INTO sesion_5.silver.fact_pedidos
SELECT
  o.order_id,
  o.customer_id,
  i.product_id,
  i.seller_id,
  TO_DATE(o.order_purchase_timestamp) AS fecha_pedido,
  o.order_status,
  i.price,
  i.freight_value AS costo_envio,
  p.payment_sequential AS secuencia_pago,
  p.payment_type AS tipo_pago,
  p.payment_installments AS cuotas_pago,
  p.payment_value AS valor_pago
FROM orders_bronze_vw o
JOIN order_items_bronze_vw i ON o.order_id = i.order_id
JOIN order_payments_bronze_vw p ON o.order_id = p.order_id;


In [0]:
%sql
-- Definiendo las Llaves Primarias para cada tabla de dimensión
-- (Usamos la tabla creada con SQL como principal para el resto del taller)
ALTER TABLE sesion_5.silver.dim_clientes_sql ADD CONSTRAINT pk_dim_clientes PRIMARY KEY(customer_id) NOT ENFORCED;
ALTER TABLE sesion_5.silver.dim_productos ADD CONSTRAINT pk_dim_productos PRIMARY KEY(product_id) NOT ENFORCED;
ALTER TABLE sesion_5.silver.dim_vendedores ADD CONSTRAINT pk_dim_vendedores PRIMARY KEY(seller_id) NOT ENFORCED;

-- Definiendo las Llaves Foráneas en la tabla de hechos
ALTER TABLE sesion_5.silver.fact_pedidos ADD CONSTRAINT fk_pedidos_clientes FOREIGN KEY(customer_id) REFERENCES sesion_5.silver.dim_clientes_sql(customer_id) NOT ENFORCED;
ALTER TABLE sesion_5.silver.fact_pedidos ADD CONSTRAINT fk_pedidos_productos FOREIGN KEY(product_id) REFERENCES sesion_5.silver.dim_productos(product_id) NOT ENFORCED;
ALTER TABLE sesion_5.silver.fact_pedidos ADD CONSTRAINT fk_pedidos_vendedores FOREIGN KEY(seller_id) REFERENCES sesion_5.silver.dim_vendedores(seller_id) NOT ENFORCED;